In [2]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image

In [3]:
my_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

file_path = './TinyImageNet'

In [5]:
label_num = 100

class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 100 * 1000

    def __getitem__(self, index):
        # print(index)
        label = index % label_num
        img_count = int(index / label_num) % 1000
        # print(f"label : {label}")
        # print(f"img_count : {img_count}")
        img = Image.open(f'{file_path}/train/{label}/{label}_{img_count}.jpg')
        # img.show()
        img = my_transform(img)
        # print(img.size())
        # print(aaa)
        return img, label

    def __len__(self):
        return self.length

In [6]:
train_loader = data.DataLoader(ImageSet(), batch_size=16, shuffle=True)
test_loader = data.DataLoader(ImageSet(), batch_size=1, shuffle=True)

In [12]:
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(256,label_num)
print(convNet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [13]:
# 加载模型， 请谨慎操作， 会覆盖在内存中的模型
convNet = models.resnet18()
convNet.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
convNet.maxpool = nn.Sequential()
convNet.layer4 = nn.Sequential()
convNet.fc = nn.Linear(256,label_num)
convNet.load_state_dict(torch.load('./ConvNet-100.model'))
convNet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Sequential()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

In [14]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.01)


In [15]:
convNet.train()
for i, (images, labels) in enumerate(train_loader):
    outputs = convNet(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    print(f"i = {i},  loss = {loss},  accuracy = {float(sum(labels == predict))/float(labels.size(0))}")

i = 0,  loss = 3.4814860820770264,  accuracy = 0.125
i = 1,  loss = 4.1566386222839355,  accuracy = 0.0625
i = 2,  loss = 3.486281633377075,  accuracy = 0.125
i = 3,  loss = 3.8409266471862793,  accuracy = 0.0625
i = 4,  loss = 4.5307111740112305,  accuracy = 0.0625
i = 5,  loss = 3.963907241821289,  accuracy = 0.0625
i = 6,  loss = 3.8952114582061768,  accuracy = 0.0
i = 7,  loss = 4.006570816040039,  accuracy = 0.0625
i = 8,  loss = 3.833636999130249,  accuracy = 0.0625
i = 9,  loss = 4.077816963195801,  accuracy = 0.0625
i = 10,  loss = 3.557100296020508,  accuracy = 0.0625
i = 11,  loss = 3.543148994445801,  accuracy = 0.125
i = 12,  loss = 4.015387058258057,  accuracy = 0.0625
i = 13,  loss = 4.384034156799316,  accuracy = 0.0
i = 14,  loss = 3.8283541202545166,  accuracy = 0.125
i = 15,  loss = 3.611889123916626,  accuracy = 0.25
i = 16,  loss = 4.175702095031738,  accuracy = 0.0
i = 17,  loss = 3.7263567447662354,  accuracy = 0.1875
i = 18,  loss = 3.9605164527893066,  accuracy 

KeyboardInterrupt: 

In [ ]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')